In [1]:
from PyPDF2 import PdfReader

In [2]:
# load pdf as text data into a variable
def load_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf = PdfReader(file)
        text = ""
        for page_num in range(len(pdf.pages)):
            page = pdf.pages[page_num]
            text += page.extract_text()
        return text

In [3]:
rubric = load_pdf('project_files/rubric.pdf')

In [4]:
def assignment_selection():
    assignment_number = int(input())
    if assignment_number not in range(2, 9):
        print("please enter a valid assignment number")
        assignment_selection()
    else:
        return load_pdf(f'project_files/questions_folder/Assignment {assignment_number}.pdf')
# print(assignment_selection())

In [5]:
#put your api key here
secret_key = " sk-proj-pqEfsvNNHPlhNAvPbJu3T3BlbkFJqfdIWBNRHYBPdTSNDYLZ"
secret_key
# Set the OpenAI API key directly as an environment variable
import os
os.environ['OPENAI_API_KEY'] = secret_key

# Import the OpenAI library
import openai

# Set the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI(api_key=secret_key)

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = prompt
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    messages.append({'role':'assistant','content': response.choices[0].message.content})
    return messages,response.choices[0].message.content

In [6]:
def py2text(file_path):
    with open(file_path, 'r') as file:
        return file.read()

In [7]:
import ipywidgets
import traitlets
print(ipywidgets.__version__)
print(traitlets.__version__)

8.1.3
5.14.3


In [8]:
import ipywidgets as widgets
from IPython.display import display
import time

global file_path
file_name = None

upload_button = widgets.FileUpload(
    accept='.py',  # Specify the file type you expect
    multiple=False  # Allow only one file to be uploaded
)

def on_upload_change(change):
    global file_path, file_name
    uploaded_file = change['owner']
    if uploaded_file.value:
        file_name = next(iter(uploaded_file.value))
        file_info = uploaded_file.value[file_name]
        file_path = f"./tempHold/{file_name}"  # Adjust the path according to your need
        with open(file_path, 'wb') as f:
            f.write(file_info['content'])
        print(f"Uploaded {file_name}")
    else:
        print("No file uploaded.")

upload_button.observe(on_upload_change, names='value')
display(upload_button)


FileUpload(value=(), accept='.py', description='Upload')

TypeError: tuple indices must be integers or slices, not Bunch

In [8]:
import os

def delete_file():
    global file_path, file_name
    
    if not file_path:
        print("Error: 'file_path' is not set.")
        return
    
    if os.path.exists(file_path):
        file_name = os.path.basename(file_path)  # Get the file name from the path
        try:
            os.remove(file_path)
            print(f"File '{file_name}' deleted successfully.")
        except Exception as e:
            print(f"Error deleting file '{file_name}': {e}")
    else:
        print(f"File '{file_path}' does not exist.")


give me a grade only based on my code delimited by triple backticks based on the rubric delimited by triple dashesand the questions delimited by triple quotes

In [9]:
def prompt_user(prev_message = None, feedback = None, insight = None):
    if insight!=None:
        prev_message.extend([{"role":"system", "content":"the student now want insight on a specific part of the code. please provide insight on the part of the code they are asking about."},
                             {"role": "user", "content": insight}])
        return get_completion(prev_message)
    if prev_message!=None:
        prev_message.extend([{"role":"system", "content":"now that you graded the student, please provide feedback to the student on what they did well and what they can improve on if they respond with yes otherwise end the conversation."},
            {"role": "user", "content": feedback}])
        return get_completion(prev_message)
    
    print("please enter the assignment number:")
    assignment = assignment_selection()
    print("please upload your student code:")
    Scode = py2text(file_path) #place holder for upload function
    prompt = [{"role": "system", "content": "You are a python teaching assistant. Your goal is to help students with their assignments. you are given code, a rubric and the questions they are solving. you will be supplying a grade only based the code, rubric and questions your initial output will only be the grade number."},
                {"role": "user", "content": f'''code: {Scode} \n\n\n assignment: {assignment} \n\n\n rubric: {rubric} \n\n\n'''}]
    return get_completion(prompt)

In [10]:
from time import sleep
def runbox():
    messages, response = prompt_user()
    print(response)
    sleep(5)
    print("would you like feedback on your code?")
    cont_cov = input("yes or no")
    if cont_cov == "yes" or cont_cov == "Yes":
        _ , feedback = prompt_user(prev_message=messages, feedback=cont_cov)
        print(feedback)
        sleep(5)
    print("would you like insight on a specific part of the code?")
    cont_cov = input("your question or no")
    if cont_cov == "no" or cont_cov == "No":
        return
    _ , insight = prompt_user(prev_message=messages, insight=cont_cov)
    print(insight)

runbox()
delete_file()

please enter the assignment number:
please upload your student code:


NameError: name 'file_path' is not defined